In [1]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

def visualize_word_regions(image_path):
    # Read the image
    if isinstance(image_path, str):
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        img = image_path.copy()
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Initialize the OCR predictor
    model = ocr_predictor(pretrained=True)
    
    # Save temporary image for doctr
    pil_image = Image.fromarray(img_rgb)
    temp_path = "temp_image.png"
    pil_image.save(temp_path)
    
    # Process with doctr
    doc = DocumentFile.from_images(temp_path)
    result = model(doc)
    
    # Create a copy for visualization
    viz_img = img_rgb.copy()
    
    # Draw boxes for each word
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                for word in line.words:
                    # Get coordinates
                    geometry = word.geometry
                    h, w = viz_img.shape[:2]
                    
                    # Convert relative coordinates to absolute pixels
                    x1 = int(geometry[0][0] * w)
                    y1 = int(geometry[0][1] * h)
                    x2 = int(geometry[1][0] * w)
                    y2 = int(geometry[1][1] * h)
                    
                    # Draw rectangle
                    cv2.rectangle(viz_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
    # Plot the results
    plt.figure(figsize=(20, 20))
    plt.imshow(viz_img)
    plt.axis('off')
    plt.title('Detected Word Regions')
    plt.show()
    
    # Clean up
    import os
    if os.path.exists(temp_path):
        os.remove(temp_path)
        
    return viz_img

ModuleNotFoundError: No module named 'cv2'